In [1]:
%pip install shap

import os
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import shap

Note: you may need to restart the kernel to use updated packages.


2025-05-29 23:26:59.172943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748561219.196676 3137598 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748561219.203924 3137598 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748561219.225269 3137598 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748561219.225297 3137598 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748561219.225300 3137598 computation_placer.cc:177] computation placer alr

In [2]:
def evaluate_classification(y_true, y_pred, results, solver_name, label):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"{label} | Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

    results.append({
        "Solver": solver_name,
        "Dataset": label,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1
    })

In [ ]:
import os
import numpy as np
import pandas as pd
import shap

def log_shap_and_importance_classification(model, X_val, y_val, pred_val, features, solver_name, target
):
    bg_idx     = np.random.choice(X_val.shape[0], min(100, X_val.shape[0]), replace=False)
    background = X_val[bg_idx]

    try:
        #  explain with DeepExplainer
        explainer   = shap.DeepExplainer(model, background)
        shap_values = explainer.shap_values(X_val)

        if isinstance(shap_values, list):
            arrs = []
            for sv in shap_values:
                a = np.array(sv)
                
                while a.ndim > 2 and 1 in a.shape:
                    a = np.squeeze(a)
                if a.ndim == 2:
                    a = a[:, :, np.newaxis]
                arrs.append(a)
            arr = np.concatenate(arrs, axis=2) 
        else:
            arr = np.array(shap_values)
            if arr.ndim == 2:
                arr = arr[:, :, np.newaxis]
            if arr.ndim == 4 and arr.shape[2] == 1:
                arr = np.squeeze(arr, axis=2)

       
        shap_2d = np.mean(arr, axis=2)           
        assert shap_2d.shape[1] == len(features)

    except Exception as e:
        print(f"SHAP failed for {solver_name}-{target}: {e}")
        return

    
    shap_df = pd.DataFrame(shap_2d, columns=features)
    shap_df["predicted_value"] = pred_val
    shap_df["actual_value"]    = y_val
    shap_df["target"]          = target
    shap_df["solver"]          = solver_name

    out_dir = "./mlp_class/mlp_shap_values"
    os.makedirs(out_dir, exist_ok=True)
    shap_df.to_csv(f"{out_dir}/shap_{solver_name}_{target}_classification.csv", index=False)

   
    feat_imp = np.abs(shap_2d).mean(axis=0)   # mean over samples
    imp_df = pd.DataFrame({
        "feature": features,
        "importance": feat_imp,
        "target": target,
        "solver": solver_name
    })

    imp_dir = "./mlp_class/mlp_feature_importance"
    os.makedirs(imp_dir, exist_ok=True)
    imp_df.to_csv(f"{imp_dir}/mlp_feature_importance_classification.csv",
                  mode='a', index=False,
                  header=not os.path.exists(f"{imp_dir}/mlp_feature_importance_classification.csv"))

    #  Top-5 features
    top5 = imp_df.nlargest(5, "importance")
    top5.to_csv(f"{imp_dir}/mlp_top5_feature_importance_classification.csv",
                mode='a', index=False,
                header=not os.path.exists(f"{imp_dir}/mlp_top5_feature_importance_classification.csv"))

    print(f"SHAP values and feature importances saved for {solver_name}-{target}.")


In [4]:
def build_mlp_classifier(input_dim, hidden_units=[128, 64], dropout_rate=0.3, num_classes=5, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(hidden_units[0], activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(dropout_rate))
    for units in hidden_units[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_mlp_classifier_for_solver(solver_name, train_file, test_file, val_file):
    print(f"\nSolver: {solver_name}")

    df_train = pd.read_csv(train_file).dropna()
    df_test  = pd.read_csv(test_file).dropna()
    df_val   = pd.read_csv(val_file).dropna()

    features = [
        "number_of_elements","capacity","max_weight","min_weight","mean_weight",
        "median_weight","std_weight","weight_range","max_profit","min_profit","mean_profit",
        "median_profit","std_profit","profit_range","renting_ratio","mean_weight_profit_ratio",
        "median_weight_profit_ratio","capacity_mean_weight_ratio","capacity_median_weight_ratio",
        "capacity_std_weight_ratio","std_weight_profit_ratio","weight_profit_correlation",
        "ram","cpu_cores"
    ]
    target_cols = ["solution_time", "optimality_gap", "peak_memory"]

    #  Load precomputed bin edges JSON
    bins_dir = os.path.join(BINS_BASE_DIR, f"{solver_name}_bins")
    if not os.path.isdir(bins_dir):
        raise FileNotFoundError(f"No bins directory: {bins_dir}")
    json_files = [f for f in os.listdir(bins_dir) if f.endswith("_bins.json")]
    if len(json_files) != 1:
        raise FileNotFoundError(f"Expected one json in {bins_dir}, found: {json_files}")
    bin_path = os.path.join(bins_dir, json_files[0])
    with open(bin_path, "r") as f:
        bin_edges_dict = json.load(f)

    scaler = StandardScaler().fit(df_train[features])
    X_train = scaler.transform(df_train[features])
    X_test  = scaler.transform(df_test[features])
    X_val   = scaler.transform(df_val[features])

 
    base_out = "./mlp_class"
    os.makedirs(f"{base_out}/mlp_classifier_models", exist_ok=True)

    results = []
    for target in target_cols:
        if target not in bin_edges_dict:
            print(f"No bin for '{target}'.")
            continue
        edges = bin_edges_dict[target]

       
        def to_bins(arr, edges):
            labels = np.digitize(arr, edges[:-1], right=False) - 1
            return np.clip(labels, 0, len(edges) - 2)

        y_train = to_bins(df_train[target].values, edges)
        y_test  = to_bins(df_test [target].values, edges)
        y_val   = to_bins(df_val  [target].values, edges)

       
        max_train = y_train.max()
        y_test = np.clip(y_test, 0, max_train)
        y_val  = np.clip(y_val,   0, max_train)

       
        if len(np.unique(y_train)) < 2:
            print(f"Skipping '{target}': only one class in train")
            continue

        # One-hot encode
        num_classes = int(max_train) + 1
        y_train_cat = to_categorical(y_train, num_classes=num_classes)
        y_val_cat   = to_categorical(y_val,   num_classes=num_classes)
        y_test_cat  = to_categorical(y_test,  num_classes=num_classes)

        #Debug 
        print(f"\n{target}: num_classes={num_classes}")
        print(" train bins:", np.unique(y_train))
        print("  val bins:", np.unique(y_val))
        print(" test bins:", np.unique(y_test))

       
        for epochs in [50, 100]:
            print(f"\nTraining MLP '{target}' ({epochs} epochs)")
            model = build_mlp_classifier(
                input_dim=X_train.shape[1],
                hidden_units=[128, 64],
                dropout_rate=0.2,
                num_classes=num_classes,
                learning_rate=0.001
            )
            model.fit(
                X_train, y_train_cat,
                validation_data=(X_val, y_val_cat),
                epochs=epochs,
                batch_size=64,
                verbose=0
            )

         
            pred_test_probs = model.predict(X_test)
            pred_val_probs  = model.predict(X_val)
            best_pred_test  = np.argmax(pred_test_probs, axis=1)
            best_pred_val   = np.argmax(pred_val_probs,  axis=1)

            print("[TEST]")
            evaluate_classification(y_test, best_pred_test, results,
                                    solver_name, f"{target} (Test) - {epochs}e")
            print("[VAL]")
            evaluate_classification(y_val,  best_pred_val,  results,
                                    solver_name, f"{target} (Val)  - {epochs}e")

            # Save model
            out_path = f"{base_out}/mlp_classifier_{solver_name}_{target}_{epochs}e.h5"
            model.save(out_path)
            print(f"Saved model {out_path}")

            # SHAP & importance
            log_shap_and_importance_classification(
                model=model,
                X_val=X_val,
                y_val=y_val,
                pred_val=best_pred_val,
                features=features,
                solver_name=solver_name,
                target=target
            )


    pd.DataFrame(results).to_csv(
        f"{base_out}/mlp_evaluation_results_classification.csv",
        mode='a', index=False,
        header=not os.path.exists(f"{base_out}/mlp_evaluation_results_classification.csv")
    )

In [6]:
def run_all_models(base_folder):
    for root, dirs, files in os.walk(base_folder):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            csv_files = os.listdir(folder_path)

            train_file = [f for f in csv_files if f.endswith("_train.csv")]
            test_file = [f for f in csv_files if f.endswith("_test.csv")]
            val_file = [f for f in csv_files if f.endswith("_val.csv")]

            if train_file and test_file and val_file:
                train_fp = os.path.join(folder_path, train_file[0])
                test_fp = os.path.join(folder_path, test_file[0])
                val_fp = os.path.join(folder_path, val_file[0])

                solver_name = folder  
                train_mlp_classifier_for_solver(solver_name, train_fp, test_fp, val_fp)

In [ ]:
base_folder = "./trainingData/final_td_min/td_models" #Path to training data
BINS_BASE_DIR = "./trainingData/final_td_min/td_bindata" #Path to bin data
run_all_models(base_folder)


Solver: or_min

=== solution_time → num_classes=2 ===
 train bins: [0 1]
  val bins: [0]
 test bins: [0]

Training MLP for 'solution_time' (50 epochs)...


/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748561275.000524 3137598 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21056 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6
I0000 00:00:1748561275.003875 3137598 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22170 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:a1:00.0, compute capability: 8.6
I0000 00:00:1748561277.273275 3138663 service.cc:152] XLA service 0x74b148004fd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748561

18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


[TEST]
solution_time (Test) - 50e | Acc: 0.9750, Prec: 0.5000, Rec: 0.4875, F1: 0.4937
[VAL]
solution_time (Val)  - 50e | Acc: 0.9750, Prec: 0.5000, Rec: 0.4875, F1: 0.4937
  • saved MLP model → ./binres_min_kp/mlp_class/mlp_classifier_or_min_solution_time_50e.h5


/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/shap/explainers/_deep/deep_tf.py:94: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn(
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(100, 24))']
  warnings.warn(msg)
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(200, 24))']
  warnings.warn(msg)
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected struc

...

SHAP values and feature importances saved for bb_min-optimality_gap.

Training MLP for 'optimality_gap' (100 epochs)...
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


[TEST]
optimality_gap (Test) - 100e | Acc: 0.8750, Prec: 0.4500, Rec: 0.6939, F1: 0.5103
[VAL]
optimality_gap (Val)  - 100e | Acc: 0.6607, Prec: 0.2293, Rec: 0.2780, F1: 0.2463
  • saved MLP model → ./binres_min_kp/mlp_class/mlp_classifier_bb_min_optimality_gap_100e.h5


/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/shap/explainers/_deep/deep_tf.py:94: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn(
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_432
Received: inputs=['Tensor(shape=(100, 24))']
  warnings.warn(msg)
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_432
Received: inputs=['Tensor(shape=(200, 24))']
  warnings.warn(msg)


SHAP values and feature importances saved for bb_min-optimality_gap.
  • skipping 'peak_memory': only one class in train


/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_432
Received: inputs=['Tensor(shape=(560, 24))']
  warnings.warn(msg)
